<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/05-%E7%89%B9%E5%BE%81%E4%BA%A4%E4%BA%92/09_NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from sequence_feature_layer import SequenceFeatures
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D, DenseFeatures

## 0.准备工作

In [ ]:
user_id = fc.sequence_categorical_column_with_hash_bucket('user_id', hash_bucket_size=10, dtype=tf.int64)
pos_item = fc.sequence_categorical_column_with_hash_bucket('pos_item', hash_bucket_size=20, dtype=tf.int64)
neg_item = fc.sequence_categorical_column_with_hash_bucket('neg_item', hash_bucket_size=20, dtype=tf.int64)

user_id_col = fc.embedding_column(user_id, dimension=8)
pos_item_col = fc.embedding_column(pos_item, dimension=8)
neg_item_col = fc.embedding_column(neg_item, dimension=8)

feature_columns = [user_id_col, pos_item_col, neg_item_col]

features={
  "user_id": tf.sparse.SparseTensor(
      indices=[[0, 0],[1, 0], [2, 0]],
      values=[1000,1001,1002],
      dense_shape=[3, 1]),
  "pos_item": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[1100, 1101, 1102, 1101, 1103],
      dense_shape=[3, 2]),
  
  "neg_item": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[2200, 2201, 2202, 2201, 2203],
      dense_shape=[3, 2]),
}
tf.sparse.to_dense(features['user_id'])

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[1000],
       [1001],
       [1002]], dtype=int32)>

# 1.MF输入

In [ ]:
mf_input_layer = SequenceFeatures(feature_columns, name='mf_input_layer')
mf_dict, mf_len_dict = mf_input_layer(features)
mf_dict.keys()

dict_keys(['neg_item_embedding', 'pos_item_embedding', 'user_id_embedding'])

## 1.1element-wise product
对应元素相乘

In [ ]:
user_emb = mf_dict['user_id_embedding']
pos_item_emb = mf_dict['pos_item_embedding']
neg_item_emb = mf_dict['neg_item_embedding']
mf_pos_output = tf.nn.sigmoid(user_emb * pos_item_emb)
mf_neg_output = tf.nn.sigmoid(user_emb * neg_item_emb)
mf_pos_output = tf.reduce_mean(mf_pos_output, axis=1)
mf_neg_output = tf.reduce_mean(mf_neg_output, axis=1)
tf.shape(mf_pos_output),tf.shape(mf_neg_output)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 8], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 8], dtype=int32)>)

# 2.Deep输入

In [ ]:
input_layer = SequenceFeatures(feature_columns, name='mf_input_layer')
seq_dict, seq_len_dict = input_layer(features)
seq_dict.keys()

dict_keys(['neg_item_embedding', 'pos_item_embedding', 'user_id_embedding'])

In [ ]:
deep_user_emb = tf.squeeze(seq_dict['user_id_embedding'], axis=1)
deep_pos_item_emb = seq_dict['pos_item_embedding']
deep_neg_item_emb = seq_dict['neg_item_embedding']

deep_pos_item_emb = tf.reduce_mean(deep_pos_item_emb, axis=1)
deep_pos_input = tf.concat([deep_user_emb, deep_pos_item_emb], axis=-1)

deep_neg_item_emb = tf.reduce_mean(deep_neg_item_emb, axis=1)
deep_neg_input = tf.concat([deep_user_emb, deep_neg_item_emb], axis=-1)
tf.shape(deep_pos_input), tf.shape(deep_neg_input)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 3, 16], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 3, 16], dtype=int32)>)

## 2.1Deep Net

In [ ]:
hidden_units = [16,8]
dnn = [Dense(units=unit, activation='relu') for unit in hidden_units]
output_layer = Dense(1, activation='sigmoid')

In [ ]:
inputs = deep_pos_input
for nn in dnn:
  inputs = nn(inputs)
deep_pos_output = inputs

In [ ]:
inputs = deep_neg_input
for nn in dnn:
  inputs = nn(inputs)
deep_neg_output = inputs

# 3.NCF输出

In [ ]:
pos_output = tf.concat([mf_pos_output, deep_pos_output], axis=-1)
neg_output = tf.concat([mf_neg_output, deep_neg_output], axis=-1)
pos_logits = output_layer(pos_output)
neg_logits = output_layer(neg_output)
pos_logits,neg_logits

(<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[0.41113794],
        [0.46307915],
        [0.48769495]], dtype=float32)>,
 <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[0.4391548 ],
        [0.44921368],
        [0.46856785]], dtype=float32)>)

In [ ]:
losses = 0.5 * tf.reduce_mean(- tf.math.log(pos_logits) - tf.math.log(1 - neg_logits))
losses

<tf.Tensor: shape=(), dtype=float32, numpy=0.6972746>